In [6]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
import collections

In [ ]:
nltk.download('rslp')
from nltk.stem import RSLPStemmer

In [8]:
data = pd.read_csv('https://github.com/dnrocha1/information_retrieval/blob/master/data/results.csv?raw=true')

# Estratégia de tokenização

#### Escolha uma estratégia de tokenização para a coleção que você está usando e justifique sua estratégia. É importante que você inclua decisões adicionais em relação ao que foi feito no Laboratório anterior (por exemplo, tratamento de maiúsculas/minúsculas e strings numéricas) (5 pts).

- CONSIDERAR PALAVRAS INDEPENDENTE DO TAMANHO, MAS REMOVER STOPWORDS
- REMOVER PALAVRAS MENORES QUE 3 (ESTRATEGIA ATUAL)

In [ ]:
texts = ''.join(data.text).lower()
regex = RegexpTokenizer(r'\b[A-zÀ-ú-\'\d]{3,}')
tokens = regex.tokenize(texts)
# tokens

# Top 50 palavras

Refaça a questão 2 do Laboratório anterior usando os tokens produzidos (5 pts).

In [21]:
tokens_freq = collections.Counter(tokens)
total_word_occurrences = len(tokens)

In [23]:
r = 1
words, frequencies, rankings, probs, rprobs = [], [], [], [], []
for k,v in tokens_freq.most_common():
    prob = (v/total_word_occurrences)*100
    words.append(k)
    frequencies.append(v)
    rankings.append(r)
    probs.append(prob)
    rprobs.append(r*prob/100)
    r += 1
    
words_statistics = pd.DataFrame()
words_statistics['Word'] = words
words_statistics['Freq'] = frequencies
words_statistics['r'] = rankings
words_statistics['Pr(%)'] = probs
words_statistics['r.Pr'] = rprobs

In [24]:
words_statistics.head(50)

,Word,Freq,r,Pr(%),r.Pr
0,que,6554,1,4.223129,0.042231
1,para,2323,2,1.496846,0.029937
2,uma,2224,3,1.433054,0.042992
3,com,2200,4,1.417590,0.056704
4,não,2049,5,1.320292,0.066015
5,por,1350,6,0.869885,0.052193
6,como,1329,7,0.856353,0.059945
7,mais,1206,8,0.777097,0.062168
8,dos,1062,9,0.684309,0.061588
9,mas,968,10,0.623739,0.062374


# Stemming nos tokens

Aplique Stemming nos tokens produzidos e encontre 10 exemplos de falsos positivos e 10 exemplos de falsos negativos. Que impacto você acha que falsos positivos e negativos, como esses, teriam no processamento de consultas? Dê exemplos. (5 pts).

- Falso positivo: duas palavras com significados diferentes reduzidas pro mesmo radical; 
  - (alarme falso) indica que uma coisa é verdade, quando na realidade não é.
  
  
- falso negativo: duas palavras que deveriam ter sido reduzidas pro mesmo mas estao em radicais diferentes
  - indica que uma condição não se aplica, não é verdade, quando ela na realidade ela acontece.

In [ ]:
# hashtable key: radical, value: palavra original
stemmer = RSLPStemmer()

hashtable = {}
for token in tokens:
    reduced = stemmer.stem(token)
    if reduced in hashtable.keys():
        if token not in hashtable[reduced]:
            hashtable[reduced].append(token)
    else:
        hashtable[reduced] = [token]

In [111]:
# verifica falsos positivos

BOLD = '\033[1m'
END = '\033[0m'

for x in sorted(hashtable, key=lambda k: len(hashtable[k]), reverse=True)[:16]:
    if len(hashtable[x]) > 1:
        print(f'{BOLD}{x}{END}: {hashtable[x]}\n')

est: ['estado', 'estava', 'este', 'esta', 'está', 'estes', 'estamos', 'estão', 'estavam', 'estados', 'estou', 'estante', 'estar', 'estada', 'estávamos', 'estáveis', 'estas', 'estima', 'estável', 'estará', 'estaria', 'estarei', 'ester', 'estaremos', 'estarmos', 'estando', 'estariam', 'esteira', 'estaríamos', 'estarem', 'estarão']

pass: ['passado', 'passou', 'passaram', 'passa', 'passava', 'passam', 'passo', 'passos', 'passar', 'passará', 'passada', 'passando', 'passassem', 'passe', 'passem', 'passagem', 'passes', 'passarei', 'passaria', 'passavam', 'passei', 'passear', 'passamos', 'passagens', 'passadas', 'passados', 'passarão', 'passarem', 'passível', 'passíveis']

sent: ['sentiu', 'sentia', 'sentem', 'sente', 'sentido', 'sentava', 'sentado', 'senti', 'sentei', 'sentar', 'sentimento', 'sentidos', 'sentimentos', 'sentirei', 'sentir', 'sentados', 'sentiam', 'sentiria', 'sentirão', 'sentada', 'sentirem', 'sentam', 'sentisse', 'sentíamos', 'sentimos', 'sentiram', 'sentindo']

cont: ['cont

Falso positivo: duas palavras com significados diferentes reduzidas pro mesmo radical;

Falsos positivos encontrados:
- `est`: ['estado', 'estava', 'este', 'esta', 'está', 'estes', 'estamos', 'estão', 'estavam', 'estados', 'estou', 'estante', 'estar', 'estada', 'estávamos', 'estáveis', 'estas', 'estima', 'estável', 'estará', 'estaria', 'estarei', 'ester', 'estaremos', 'estarmos', 'estando', 'estariam', 'esteira', 'estaríamos', 'estarem', 'estarão']

- trabalh
- volt
- continu
- cri
- dev
- entr

- tom TOMÁS
- fal FALENCIA, FALIDO


In [112]:
# verifica falsos negativos

for x in sorted(hashtable):
    if len(hashtable[x]) > 1:
        print(f'{BOLD}{x}{END}: {hashtable[x]}\n')
#         print('------')
#         print(x)
#         print(hashtable[x])

abal: ['abalado', 'abalaram', 'abalando', 'abalou', 'abalos', 'abalados']

abandon: ['abandonados', 'abandonado', 'abandonar', 'abandonem', 'abandonam', 'abandono', 'abandona', 'abandonando', 'abandone', 'abandonou', 'abandonaria', 'abandonarão']

abat: ['abatem', 'abateu', 'abatimentos']

abert: ['aberta', 'abertura', 'abertamente', 'aberto', 'abertos', 'abertas']

abism: ['abismos', 'abismo']

abon: ['abono', 'abonar']

abord: ['abordagens', 'aborda', 'abordou', 'abordar', 'abordado', 'aborde', 'abordados', 'abordagem', 'abordadas']

abort: ['aborto', 'abortamento']

abr: ['abrantes', 'abraão', 'abrindo', 'abriram', 'abrisse', 'abrir', 'abriu', 'abre', 'abririam', 'abrams', 'abreu', 'abrimos']

abrang: ['abrangia', 'abrange', 'abrangem', 'abrangentes']

abraç: ['abraçou', 'abraçaria', 'abraçam', 'abraçaram', 'abraços', 'abraço']

abrig: ['abrigo', 'abrigos', 'abrigar', 'abrigava', 'abriga', 'abrigaram']

abrupt: ['abruptas', 'abrupta']

absolut: ['absoluto', 'absoluta', 'absolutament


revolt: ['revolta', 'revoltados', 'revoltaram', 'revoltas']

revolucion: ['revolucionários', 'revolucionário', 'revolucionou', 'revolucionará', 'revolucionando']

rey: ['reyes', 'reys']

rez: ['rezamos', 'rezar', 'rezavam', 'reza']

reún: ['reúne', 'reúnem']

ri: ['riem', 'riam']

rib: ['ribeiro', 'rib']

ribe: ['ribeirinhos', 'ribeirinhas']

ric: ['ricos', 'rica', 'rico', 'ricas']

richard: ['richard', 'richards']

rifl: ['rifles', 'rifle']

rigor: ['rigor', 'rigorosas', 'rigorosa', 'rigoroso', 'rigores', 'rigorosos']

rio: ['rio', 'rios']

riqu: ['riqueza', 'riquezas']

rir: ['rir', 'rirem']

ris: ['risos', 'risadas', 'risível', 'riso']

risc: ['risco', 'riscos', 'risca']

rit: ['rituais', 'rito', 'ritos', 'rita']

ritm: ['ritmo', 'ritmos']

rival: ['rival', 'rivais', 'rivalidades']

robert: ['robert', 'roberto', 'roberta']

robust: ['robusto', 'robusta']

robô: ['robôs', 'robô']

rod: ['rodar', 'rodada', 'rodas', 'rodadas', 'rod', 'rodou', 'rodado', 'roda']

rode: ['rodeia', 'rodea

falso negativo: duas palavras que deveriam ter sido reduzidas pro mesmo mas estao em radicais diferentes

- 

# Questão 4

Refaça a questão 3 do Laboratório anterior usando os tokens stemizados. Você percebeu alguma diferença em relação aos tokens sem stemming? Se sim, quais? (10 pts.)